<a href="https://colab.research.google.com/github/JeJe620/NYU-SUMMER-2021/blob/main/Economics_Milestone4_IV2SLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Briefings**

1. Two packages:


*   https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html
*   https://bashtage.github.io/linearmodels/doc/iv/examples/advanced-examples.html

2. Four variables:

Dependent Variables: Y
Endogenous Variable: X
Control variables: W
instruments: Z

3. Two regressions


*   First: X~Z
*   Second: Y~(X+W)





background readings: https://cryptobriefing.com/earning-20-apy-with-yield-app/

# Data Preparations

### Date Sources 
https://docs.defipulse.com/overview-of-api-endpoints

https://data-beta.defipulse.com/dashboard/egs

https://docs.defipulse.com/api-docs-by-provider/defi-pulse-data/total-value-locked/total-value-locked

### Import the Historical Data of Total Value Locked

In [41]:
import pandas as pd
import requests
import json 
from datetime import datetime
TVL= requests.get('https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory?resolution=history&project=all&api-key=10095a3e336c5beb50c0e30ef3ab8ebdcde3e0c08f65f3ae462f69a6902a')
TVL= TVL.json()
TVL=pd.DataFrame(TVL)
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI
0,1623819600,62286443308,2.467112e+07,175577.557569,8.784443e+06,3.108168e+09
1,1623801600,62797516969,2.466565e+07,175532.268868,8.779422e+06,3.094556e+09
2,1623715200,62064623007,2.404282e+07,175064.886165,8.795703e+06,2.954794e+09
3,1623628800,60890181674,2.425498e+07,174891.430036,8.850109e+06,2.958831e+09
4,1623542400,57790475940,2.436638e+07,176037.991728,8.855824e+06,2.906476e+09


### Epoch & Unix Timestamp Conversion Tools: 

https://www.epochconverter.com/

In [42]:
import datetime
TVL['timestamp']=TVL['timestamp'].astype(int)
TVL.dtypes

timestamp      int64
tvlUSD         int64
tvlETH       float64
BTC          float64
ETH          float64
DAI          float64
dtype: object

In [43]:
TVL['time']=TVL['timestamp'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).replace(tzinfo=datetime.timezone.utc))
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time
0,1623819600,62286443308,2.467112e+07,175577.557569,8.784443e+06,3.108168e+09,2021-06-16 05:00:00+00:00
1,1623801600,62797516969,2.466565e+07,175532.268868,8.779422e+06,3.094556e+09,2021-06-16 00:00:00+00:00
2,1623715200,62064623007,2.404282e+07,175064.886165,8.795703e+06,2.954794e+09,2021-06-15 00:00:00+00:00
3,1623628800,60890181674,2.425498e+07,174891.430036,8.850109e+06,2.958831e+09,2021-06-14 00:00:00+00:00
4,1623542400,57790475940,2.436638e+07,176037.991728,8.855824e+06,2.906476e+09,2021-06-13 00:00:00+00:00


In [44]:
TVL['date'] = pd.to_datetime(TVL['time']).dt.date
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time,date
0,1623819600,62286443308,2.467112e+07,175577.557569,8.784443e+06,3.108168e+09,2021-06-16 05:00:00+00:00,2021-06-16
1,1623801600,62797516969,2.466565e+07,175532.268868,8.779422e+06,3.094556e+09,2021-06-16 00:00:00+00:00,2021-06-16
2,1623715200,62064623007,2.404282e+07,175064.886165,8.795703e+06,2.954794e+09,2021-06-15 00:00:00+00:00,2021-06-15
3,1623628800,60890181674,2.425498e+07,174891.430036,8.850109e+06,2.958831e+09,2021-06-14 00:00:00+00:00,2021-06-14
4,1623542400,57790475940,2.436638e+07,176037.991728,8.855824e+06,2.906476e+09,2021-06-13 00:00:00+00:00,2021-06-13


In [45]:
TVL=TVL.drop(columns='time')
TVL['date']=pd.to_datetime(TVL['date'])
TVL.dtypes

timestamp             int64
tvlUSD                int64
tvlETH              float64
BTC                 float64
ETH                 float64
DAI                 float64
date         datetime64[ns]
dtype: object

### Import the data for Ethereum Prices

[Data Metrics](https://github.com/coinmetrics-io/data/blob/master/csv/metrics.csv)

In [46]:
df_eth=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/eth.csv",usecols=(["time","PriceUSD","VtyDayRet30d","TxTfrValAdjUSD","AdrBalCnt","NVTAdj90","SplyCur"]))
df_eth['time']=pd.to_datetime(df_eth['time'])
df_eth.dtypes

time              datetime64[ns]
AdrBalCnt                float64
NVTAdj90                 float64
PriceUSD                 float64
SplyCur                  float64
TxTfrValAdjUSD           float64
VtyDayRet30d             float64
dtype: object

### Merge the two Data Frame

In [47]:
df =df_eth.merge(TVL, left_on='time', right_on='date')
df.head()

,time,AdrBalCnt,NVTAdj90,PriceUSD,SplyCur,TxTfrValAdjUSD,VtyDayRet30d,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,date
0,2017-08-02,2563959.0,20.362229,217.584664,9.374206e+07,6.780871e+08,0.095886,1501632000,0,0.000000,0.0,0.0,0.0,2017-08-02
1,2017-08-03,2583277.0,20.979627,224.028898,9.376303e+07,6.062566e+08,0.096060,1501718400,4,0.018339,0.0,0.0,0.0,2017-08-03
2,2017-08-04,2605667.0,20.714810,221.471094,9.378433e+07,5.206240e+08,0.096060,1501804800,4,0.017826,0.0,0.0,0.0,2017-08-04
3,2017-08-05,2624386.0,23.205816,248.939666,9.380552e+07,6.665590e+08,0.098653,1501891200,5,0.022665,0.0,0.0,0.0,2017-08-05
4,2017-08-06,2640032.0,24.667311,266.031146,9.382679e+07,9.229966e+08,0.097469,1501977600,5,0.019756,0.0,0.0,0.0,2017-08-06


In [48]:
df=df[["date","tvlUSD","VtyDayRet30d","SplyCur","AdrBalCnt"]]

# Test Instrument Validities

Y: "SplyCur"

X: "VtyDayRet30d"

Z: "SplyCur"

W: "AdrBalCnt"

In [49]:
df[["VtyDayRet30d","SplyCur"]].corr()

,VtyDayRet30d,SplyCur
VtyDayRet30d,1.000000,-0.117955
SplyCur,-0.117955,1.000000


# Using the Linear Model Package

https://bashtage.github.io/linearmodels/doc/iv/index.html

In [50]:
pip install linearmodels

In [51]:
from linearmodels import IV2SLS

### run an OLS

In [52]:
df['const'] = 1

In [53]:
ivolsmod = IV2SLS(df.tvlUSD, df[["const","VtyDayRet30d","SplyCur","AdrBalCnt"]], None, None)
res_ols = ivolsmod.fit()
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:                 tvlUSD   R-squared:                      0.7298
Estimator:                        OLS   Adj. R-squared:                 0.7292
No. Observations:                1413   F-statistic:                    1314.0
Date:                Wed, Jun 16 2021   P-value (F-stat)                0.0000
Time:                        07:14:11   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const         6.861e+11  2.542e+10     26.987     0.0000   6.363e+11    7.36e+11
VtyDayRet30d  6.697e+10  1.256e+10     5.331

### run an IV2SLS

In [54]:
ivmod = IV2SLS(df.tvlUSD, df[["const","AdrBalCnt"]], df["VtyDayRet30d"], df["SplyCur"])
res_2sls = ivmod.fit()
print(res_2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 tvlUSD   R-squared:                     -3.8811
Estimator:                    IV-2SLS   Adj. R-squared:                -3.8880
No. Observations:                1413   F-statistic:                    204.93
Date:                Wed, Jun 16 2021   P-value (F-stat)                0.0000
Time:                        07:14:11   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const        -1.183e+11  1.346e+10    -8.7840     0.0000  -1.446e+11  -9.187e+10
AdrBalCnt        876.87     63.609     13.78

### comparison of the results

In [55]:
from collections import OrderedDict
from linearmodels.iv.results import compare

In [56]:
res = OrderedDict()
res['OLS'] = res_ols
res['2SLS'] = res_2sls
print(compare(res))

                 Model Comparison                
                               OLS           2SLS
-------------------------------------------------
Dep. Variable               tvlUSD         tvlUSD
Estimator                      OLS        IV-2SLS
No. Observations              1413           1413
Cov. Est.                   robust         robust
R-squared                   0.7298        -3.8811
Adj. R-squared              0.7292        -3.8880
F-statistic                 1314.0         204.93
P-value (F-stat)            0.0000         0.0000
==================     ===========   ============
const                    6.861e+11     -1.183e+11
                          (26.987)      (-8.7840)
VtyDayRet30d             6.697e+10      1.958e+12
                          (5.3313)       (7.8673)
SplyCur                    -7437.2               
                         (-27.468)               
AdrBalCnt                   3595.2         876.87
                          (29.542)       (13.785)


### Testing endogeneity

The Durbin test is a classic of endogeneity which compares OLS estimates with 2SLS and exploits the fact that OLS estimates will be relatively efficient. Durbin’s test is not robust to heteroskedasticity.

In [57]:
res_2sls.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 649.7442
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2d772ee850

Wooldridge’s regression-based test of exogeneity is robust to heteroskedasticity since it inherits the covariance estimator from the model. Here there is little difference.

In [58]:
res_2sls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 754.5060
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f2d808787d0

### print first stage regression

In [59]:
print(res_2sls.first_stage)

     First Stage Estimation Results    
                           VtyDayRet30d
---------------------------------------
R-squared                        0.0562
Partial R-squared                0.0499
Shea's R-squared                 0.0499
Partial F-statistic              64.337
P-value (Partial F-stat)      9.992e-16
Partial F-stat Distn            chi2(1)
========================== ============
const                            0.4254
                               (9.1767)
AdrBalCnt                     1.437e-09
                               (7.4958)
SplyCur                      -3.933e-09
                              (-8.0211)
---------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


# Using the Statsmodel Package

Not recommended

https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html